In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
EPOCH = 200
LR = 0.1
BATCH_SIZE = 100

In [4]:
class Classification(nn.Module):
    def __init__(self):
        super(Classification, self).__init__()
        self.linear1 = nn.Linear(2,10)
        self.linear2 = nn.Linear(10,6, bias=True)
        self.linear3 = nn.Linear(6,2)
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.softmax(self.linear3(x))
        return x

In [5]:
if torch.cuda.is_available():
    model = Classification().cuda()
else:
    model = Classification()
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = LR)

In [6]:
# preparing the dataset
x = torch.unsqueeze((torch.rand(1000)-0.5)*10,dim=0).reshape(-1,2)
y = torch.LongTensor([1 if _x[0]*_x[1]>=0 else 0 for _x in x])

In [34]:
from torch.utils.data import TensorDataset, DataLoader
xy_dataset = TensorDataset(x,y)
data_loader = DataLoader(dataset=xy_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         num_workers=2)

In [35]:
s = int(len(data_loader)*0.8)
e = len(data_loader)-s
train_dataset, test_dataset = torch.utils.data.random_split(data_loader,[s,e])

In [12]:
for epoch in range(EPOCH):
    for i,batch in enumerate(train_dataset.dataset):
        x,y = batch
        if torch.cuda.is_available():
            x = torch.autograd.Variable(x).cuda()
            y = torch.autograd.Variable(y).cuda()
        else:
            x = torch.autograd.Variable(x)
            y = torch.autograd.Variable(y)
        out = model(x)
    #     print(out.shape,y.shape)
        loss = loss_func(out,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch+1)%100==0:
        print('Epoch[{}/{}],loss:{:.6f}'.format(epoch+1,EPOCH,loss.data.item()))

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


Epoch[100/200],loss:0.339697
Epoch[200/200],loss:0.328606


In [19]:
from sklearn.metrics import accuracy_score

In [31]:
model.eval()
acc = 0.0
for i,batch in enumerate(test_dataset.dataset):
    x,y=batch
    if  torch.cuda.is_available():
        x = x.cuda()
    predict = model(x)
    predict = torch.max(predict,1)[1]
    predict = predict.cpu().data.numpy()
    acc += accuracy_score(y.cpu().numpy(),predict)
print('{:.3f}'.format(acc/(i+1)))

0.996


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.
